In [25]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
import json

import lxml.html as lh

print('Libraries imported.')

Libraries imported.


In [14]:
url= "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
print(url)

https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M


In [26]:
#Create a handle, page, to handle the contents of the website
page = requests.get(url)
#Store the contents of the website under doc
doc = lh.fromstring(page.content)
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

In [28]:
[len(T) for T in tr_elements[:12]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [34]:
tr_elements = doc.xpath('//tr')
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print (i,name)
    col.append((name,[]))

1 Postcode
2 Borough
3 Neighbourhood



In [35]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 10, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [37]:
[len(C) for (title,C) in col]

[288, 288, 288]

In [83]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)
df

,Borough,Neighbourhood,Postcode
0,Not assigned,Not assigned\n,M1A
1,Not assigned,Not assigned\n,M2A
2,North York,Parkwoods\n,M3A
3,North York,Victoria Village\n,M4A
4,Downtown Toronto,Harbourfront\n,M5A
5,Downtown Toronto,Regent Park\n,M5A
6,North York,Lawrence Heights\n,M6A
7,North York,Lawrence Manor\n,M6A
8,Queen's Park,Not assigned\n,M7A
9,Not assigned,Not assigned\n,M8A


In [84]:
df.columns = df.columns.str.strip()
df['Neighbourhood'] = df['Neighbourhood'].str.replace('/n', '')

In [66]:
df.columns = df.columns.str.strip()

In [85]:
df['Neighbourhood']

0                                         Not assigned\n
1                                         Not assigned\n
2                                            Parkwoods\n
3                                     Victoria Village\n
4                                         Harbourfront\n
5                                          Regent Park\n
6                                     Lawrence Heights\n
7                                       Lawrence Manor\n
8                                         Not assigned\n
9                                         Not assigned\n
10                                    Islington Avenue\n
11                                               Rouge\n
12                                             Malvern\n
13                                        Not assigned\n
14                                     Don Mills North\n
15                                    Woodbine Gardens\n
16                                       Parkview Hill\n
17                             

In [73]:
df['Neighbourhood'] = df['Neighbourhood'].str.replace('\n', '')

In [86]:
df = df[['Postcode','Borough','Neighbourhood']]
df

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n
5,M5A,Downtown Toronto,Regent Park\n
6,M6A,North York,Lawrence Heights\n
7,M6A,North York,Lawrence Manor\n
8,M7A,Queen's Park,Not assigned\n
9,M8A,Not assigned,Not assigned\n


In [87]:
df = df.set_index("Borough")
df = df.drop("Not assigned", axis=0)

In [80]:
df= df.set_index("Postcode")

In [89]:
df.reset_index(level=0, inplace=True)

In [91]:
df=  df[['Postcode','Borough','Neighbourhood']]

In [93]:
df['Neighbourhood'] = df['Neighbourhood'].str.replace('\n', '')

In [104]:
df2 = df
df2.groupby('Postcode')
for i in df2
df2['Neighbourhood'].drop('Not assigned',axis=0)

ValueError: labels ['Not assigned'] not contained in axis